<a href="https://colab.research.google.com/github/AnaliaLeyez/AnaliaLeyez/blob/main/u3_limpieza_parte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UNIDAD III - Limpieza y preparación

El notebook esta basado en el contenido del libro Python for Data Analysis.
[Chapter 7 Data Cleaning and Preparation. (Wes McKinney)](https://wesmckinney.com/book/data-cleaning)



# Inicializaciones

## Importamos dependencias

In [ ]:
import numpy as np
import pandas as pd


### Parametros por defecto

In [ ]:
np.random.seed(12345)

# Carga y diagnóstico base

In [ ]:
# Dataset sintético para practica
practica = pd.DataFrame({
    'id':[1,1,2,3,4,5],
    'nombre':[' Ana ', 'ana', 'TOM', 'Luz', None, '   Marta'],
    'monto':['1.234,00','500,50', np.nan, '100,0', '10.000,99', 'quince'],
    'categoria':['A','a','B','B','C','C '],
    'fecha':['2024-01-10','2024/01/11','2055-01-01', None, '01-02-2024', '2024-02-03'],
})
practica

## Tamaño del dataset

In [ ]:
# Nro de filas y columnas)
practica.shape

## Estructura del dataset

In [ ]:
#Resumen estándar
practica.info()

## Estadísticas descriptivas

In [ ]:
#(count:cantidad de valores no nulos),mean: media o promedio.std: desviación estándar (dispersión).
# min, quartiles, max) de columnas numéricas para ver rangos/outliers
practica.describe ()

## 2) **Tipificación**: convertir tipos (números/fechas)

Muchos datasets vienen con formato local distinto al que utilizamos (puntos de miles y coma decimal) y fechas como texto. Si no tipificamos, puede ocurrir que Pandas trate esos campos como strings. Por lo cual el objetivo de la tipificacion es llevar cada columna a su tipo correcto (numérico/fecha) para poder analizar sin errores y con buena performance.

In [ ]:
## trabajamos sobre una copia para conservar el original
df = practica.copy()

# 1) Normalizar números en formato “AR/ES” (miles='.', decimales=',') → formato estándar con punto decimal
# astype(str) pone toda la columna en formato texto para poder usar str de forma segura y vectorizada
df['monto'] = (df['monto'].astype(str)
                .str.replace('.', '', regex=False)  # quita separadores de miles
                .str.replace(',', '.', regex=False))# convierte la coma decimal a punto

# 2) Convertir a numérico y los valores inválidos a NaN
df['monto'] = pd.to_numeric(df['monto'], errors='coerce')

# 3) Parsear fechas desde texto (día/mes primero) a datetime y los valores inválidos a NaT
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce', dayfirst=True)

# 4) Ajustar tipos “óptimos” de pandas (Int64, string, boolean, etc.)
df = df.convert_dtypes()

df.dtypes  # verificación: confirma tipos finales por columna

# Datos faltantes / perdidos

¿Por qué importan y cómo los tratamos?

En datasets reales es habitual que falten valores (no medidos, errores de carga, formatos inconsistentes). Ignorarlos distorsiona análisis y rompe modelos.

Por ello nuestro objetivo es detectar, hacer visibles los faltantes (muchas veces aparecen recién al tipificar números/fechas) y decidir un tratamiento coherente con el problema.

Causas típicas:

- No se midió / no aplica.

- Error de captura o formato (“quince”, fecha inválida).

- Integración de fuentes heterogéneas.

¿Còmo tratarlos?:

1-Diagnosticar.

2-Tipificar: convertir números y fechas para revelar NA ocultos.

3-Decidir: imputar (mediana/0/forward fill), dejar NA o descartar filas/columnas según impacto.

4-Validar: revisar estadísticas antes/después

5-Documentar: anotar por qué se eligió cada criterio.

**Importante hay que decidir con criterio, no “por defecto”; diferenciá el cero de desconocido y conservá una versión original para comparar los cambios.**


`isna()` retorna una serie booleana con True donde hay valores nulos (`NaN` o `None`)

In [ ]:
# ¿Dónde faltan datos?
practica.isna()

In [ ]:
  # ¿Dónde faltan datos? máscara booleana fila a fila
practica['fecha'].isna()

In [ ]:
# ¿Cuantos por columna?
practica.isna().sum()

In [ ]:
# ¿Que % por columna?
practica[['nombre','monto','fecha']].isna().mean().round(2)

In [ ]:
# ¿que columnas tinen todos NA?
practica.isna().all()

In [ ]:
# que columnas tienen al menos algun NA
practica.isna().any()

In [ ]:
practica

## Como encontramos valores NA

In [ ]:
# que columnas tienen al menos algun NA
practica.isna().any()

In [ ]:
#que filas tienen todos NA
practica.isna().all(axis=1)

In [ ]:
# que fila tienen al menos algun NA
practica.isna().any(axis=1)

In [ ]:
# cuantos na hay en cada fila
practica.isna().sum(axis="columns")

## Filtrar datos perdidos (dropna)

Existe dos metodos utiles para manipular datos perdidos o no disponibles

* dropna() -> sirve para **quitar** datos cuando hay datos "no disponibles" (NA o perdidos)
* fillna() -> sirve para **completar** cuando hay datos perdidos

`dropna` retorna un nuevo dataframe filtrando los valores null

In [ ]:
practica.dropna()

Existen muchas opciones para controlar el comportamiento de [`dropna`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) en series y en dataframe

In [ ]:
# armamos un dataset con varios datos NA
data = pd.DataFrame([[1., 6.5, 3.],
                     [1., np.nan, np.nan],
                     [np.nan, np.nan, np.nan],
                     [np.nan, 6.5, 3.]])

data.columns = ["A", "B", "C"]

data

In [ ]:
# quita todos los renglones con algun NA (not available)
data.dropna()

Con `how` definimos como quitar.
* "any" quita la fila si al menos hay algun NA. (default)
* "all" quita solo si todos son NA

In [ ]:
data.dropna(how="all")

Parametro `axis` indica que eliminar.

* 'index' o 0 : remueve renglones
* 'columns' o 1 : remueve columnas.

In [ ]:
# agregamos una columna toda NA
data["D"] = np.nan

# mostramos
data

In [ ]:
# removemos la unica columna que tiene todos sus valores NA
data.dropna(axis="columns", how="all")

El parámetro `tresh` indica desde cuando opera `dropna`. `dropna` funciona cuando la cantidad de NA son mayor o igual a `tresh`

In [ ]:
#Df para ejemplo
df = pd.DataFrame(np.random.standard_normal((7, 3)))
df

In [ ]:
# agregamos algunos NaN
df.iloc[0:4, 1] = np.nan
df.iloc[0:2, 2] = np.nan
df

In [ ]:
df.dropna()

In [ ]:
# quita cuando hay dos o mas Nan
df.dropna(thresh=2)

In [ ]:
# dropear cuando hay nan en la serie 2
df.dropna(subset=[2])

## Llenar datos perdidos (fillna)

fillna se puede usar para llenar / completar los datos perdidos.

In [ ]:
# armamos dataframe de ejemplo
df = pd.DataFrame(np.random.standard_normal((7, 3)))
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df

`fillna` retorna un DataFrame llenando cualquier valor NA con el valor especificado

In [ ]:
df.fillna(0)

Podemos hacer que funcione con ciertos valores para cada columna

In [ ]:
# usar 0.5 para la segunda columna y 0.0 para la última
df.fillna({1: 0.5, 2: 7})

In [ ]:
# renobramos las columnas
df.columns = ["A", "B", "C"]
df

In [ ]:
# Lo mas comun es tener series con nombres de strings
df.fillna({"B": 0.5, "C": 0})

El parámetro `method` permite definir con que valores completa.
* `ffill` completa el valor con el último valor del eje 'yendo hacia adelante'.
* `bfill` completa el valor con el último valor del eje 'yendo hacia atras'.


In [ ]:
# Creamos Df de ejemplo y le agregamos NaN
df = pd.DataFrame(np.random.standard_normal((6, 3)), columns=["A", "B", "C"])
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

In [ ]:
#completa el valor con el último valor del eje 'yendo hacia adelante'.
df.ffill()

Podemos ffill y bfill para completar datos de solo ciertas columnas usando indexación de los dataframe

In [ ]:
df["B"]

In [ ]:
#Completamos varias columnas (por columna, vertical)
df[["B","C"]].ffill()

Podemos usar los valores estadísticos para completar los NAs.

Por ejemplo, completar los NA de una `Serie`| con su **media**.

In [ ]:
#creamos serie para el ejemplo
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data

In [ ]:
#Rellena los NaN con la media de la serie (la media se calcula ignorando los NaN).
data.fillna(data.mean())

## Transformar con inplace

In [ ]:
# armamos dataframe de ejemplo
df = pd.DataFrame(np.random.standard_normal((6, 3)))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

In [ ]:
#Eliminar filas con 2 valores NO nulos
df.dropna(thresh=2, inplace=True)
df

# Transformaciones

## Quitar duplicados

A continuación un conjunto de funciones de pandas utiles para poder transformar los datos con el fin de limpiarlos y preparlos.

In [ ]:
# DataFrame de ejemplo
df = pd.DataFrame({
    "k1": ["A","A","B","B","B","C"],
    "k2": [ 1 , 1 , 1 , 2 , 2 , 3 ]
})
print(df)

El método `duplicate` sirve para encontrar renglones repetidos que tengan valores repetidos en todas o algunas columnas. Si hay varios duplicados, mantiene el primero y marca el resto como duplicados.

In [ ]:
df.duplicated()
#devuelve una Serie booleana: False en la 1ª aparición, True en repeticiones

`drop_duplicates` sirve para elimnar los duplicados. Si hay varios duplicados, mantiene el primero y elimina el resto como duplicados.

In [ ]:
df.drop_duplicates()

Podemos elegir quedarnos con los últimos duplicados

In [ ]:
df.drop_duplicates(subset=["k1"], keep='last')

## Quitar ejes

Método `drop`


In [ ]:
k1 = ["one", "two"] * 3 + ["two"]
k2 = [1, 1, 2, 3, 3, 4, 4]
v1 = [0, 1, 2, 3, 4, 5, 6]

data = pd.DataFrame({"k1": k1,
                     "k2": k2,
                     "v1": v1 })
data

Eliminar filas por su `index`

In [ ]:
data.drop(index=[0,2,4,6])

Eliminar columnas

In [ ]:
data.drop(columns=["v1"])

## Remplazar valores

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3. ,2, 4, 7])
data

Otra manera de realizar transformaciones es con [`replace`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html)

Supongamos que detectamos valores extremos que no corresponde a una medición con sentido.

In [ ]:
# podemos reemplazar de aun valor
data.replace(-999, np.nan)

In [ ]:
# si pasamos una lista podemos reemplazar varios valores en unico valor
data.replace([-999, -1000], np.nan)

In [ ]:
# podemos reemplazar varios valores y asignar un con una lista el valor
# por el cual reemplazar
data.replace([-999, -1000], [np.nan, 0])

## Renombrar indices y columnas

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])
data

Podemos cambiarlo obteniendo el indice y luego modificarlo

In [ ]:
# Este es el indice original
data.index

In [ ]:
def transform(x):
    return x[:4].upper() #toma los 4 primeros caracteres y los convierte a MAYÚSCULAS

data.index.map(transform) #aplica 'transform' a cada etiqueta del índice y devuelve un NUEVO Index

In [ ]:
# map retorna un nuevo index. Ahora actualizamos el nuevo indice
data.index = data.index.map(transform)
data

Con `rename` podemos cambiar los indices y las columnas de manera más concisa y flexible. Ver documentación [rename](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html)  

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])
data

In [ ]:
# o podemos pasar un diccionario
data.rename(index={"Ohio": "Indiana"},
           o" columns={"three": "peekabo})

In [ ]:
# podemos pasar funciones para que realicen el cambio
data.rename(index=str.title, columns=str.upper)
#El index se transforman con str.title (cada palabra en Title, 1ra letra mayuscula).
#Las columnas se transforman con str.upper (MAYÚSCULAS).

`rename` no modifica los valores del dataframe, para eso podemos usar la opcion `inplace=True`

In [ ]:
data.rename(index=str.title, columns=str.upper, inplace=True)
data